# 02 - Modèles Baseline pour la Classification de Radiographies COVID-19

**Objectif**: Implémenter et évaluer des modèles baseline (Logistic Regression, Random Forest, SVM) pour la classification de radiographies thoraciques.

**Features utilisées**: 
- `mean_intensity` : Intensité moyenne des pixels
- `std_intensity` : Écart-type de l'intensité
- `contrast` : Ratio std/mean
- `entropy` : Entropie de Shannon (complexité de texture)
- `gradient` : Gradient Sobel moyen (intensité des contours)

In [ ]:
# Imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Add src to path for local imports
sys.path.insert(0, os.path.abspath('..'))

from src.features.build_features import build_feature_dataset, load_or_build_features
from src.models.train_model import train_all_models, get_feature_importance, FEATURE_COLS
from src.models.predict_model import plot_confusion_matrix, plot_feature_importance, generate_report

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
%matplotlib inline

print("Imports OK")

## 1. Chargement des Features

Chargement ou extraction des 5 features statistiques (voir `01-projet.ipynb` pour l'analyse exploratoire).

In [ ]:
# Configuration des chemins
# ⚠️ MODIFIER CE CHEMIN SI NÉCESSAIRE
DATA_DIR = "../data/COVID-19_Radiography_Dataset"
FEATURES_CACHE = "../data/features.csv"
MODELS_DIR = "../models"
REPORTS_DIR = "../reports"

os.makedirs(os.path.join(REPORTS_DIR, 'figures'), exist_ok=True)

# Extraction des features (ou chargement depuis le cache)
df_features = load_or_build_features(DATA_DIR, cache_path=FEATURES_CACHE)

print(f"\nDataset shape: {df_features.shape}")
df_features.head()

## 2. Entraînement des Modèles Baseline

Nous allons entraîner 3 modèles baseline:
1. **Logistic Regression** - Baseline linéaire simple
2. **Random Forest** - Méthode ensemble robuste
3. **SVM (RBF Kernel)** - Classifieur non-linéaire

In [ ]:
# Entraînement de tous les modèles baseline
# Cross-validation 5-fold + évaluation sur test set (20%)

results = train_all_models(
    df=df_features,
    save_dir=MODELS_DIR
)

## 3. Évaluation et Analyse des Résultats

In [ ]:
# Tableau comparatif des performances
comparison_data = []
for name, res in results.items():
    comparison_data.append({
        'Modèle': name,
        'CV Accuracy': f"{res['train']['cv_mean']:.4f} ± {res['train']['cv_std']:.4f}",
        'Test Accuracy': f"{res['eval']['accuracy']:.4f}",
        'F1 Weighted': f"{res['eval']['f1_weighted']:.4f}",
        'Precision': f"{res['eval']['precision_weighted']:.4f}",
        'Recall': f"{res['eval']['recall_weighted']:.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df

In [ ]:
# Matrices de confusion pour chaque modèle
le = joblib.load(os.path.join(MODELS_DIR, 'label_encoder.joblib'))
class_names = le.classes_

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, res) in enumerate(results.items()):
    cm = res['eval']['confusion_matrix']
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names, ax=axes[idx])
    axes[idx].set_title(f'{name}\nAccuracy: {res["eval"]["accuracy"]:.4f}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(os.path.join(REPORTS_DIR, 'figures', 'confusion_matrices_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature Importance (Random Forest)
importance_df = get_feature_importance(os.path.join(MODELS_DIR, 'random_forest.joblib'))

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(importance_df['feature'], importance_df['importance'], color='steelblue')
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Random Forest - Importance des Features', fontsize=14, fontweight='bold')

for bar, val in zip(bars, importance_df['importance']):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center')

plt.tight_layout()
plt.savefig(os.path.join(REPORTS_DIR, 'figures', 'feature_importance.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Top features:")
importance_df.sort_values('importance', ascending=False)

In [ ]:
# Métriques par classe - Meilleur modèle
best_model_name = max(results.keys(), key=lambda x: results[x]['eval']['f1_weighted'])
best_result = results[best_model_name]

print(f"Meilleur modèle: {best_model_name}")
print(f"F1 Weighted: {best_result['eval']['f1_weighted']:.4f}\n")

class_report = best_result['eval']['classification_report']
metrics_data = []
for class_name in class_names:
    if class_name in class_report:
        cr = class_report[class_name]
        metrics_data.append({
            'Classe': class_name,
            'Precision': f"{cr['precision']:.4f}",
            'Recall': f"{cr['recall']:.4f}",
            'F1-Score': f"{cr['f1-score']:.4f}",
            'Support': int(cr['support'])
        })

pd.DataFrame(metrics_data)

## 4. Conclusions et Interprétations

In [ ]:
# Sauvegarde des résultats
comparison_df.to_csv(os.path.join(REPORTS_DIR, 'baseline_comparison.csv'), index=False)

best_acc = max(results.values(), key=lambda x: x['eval']['accuracy'])['eval']['accuracy']
best_f1 = max(results.values(), key=lambda x: x['eval']['f1_weighted'])['eval']['f1_weighted']
lr_acc = results['Logistic Regression']['eval']['accuracy']
rf_acc = results['Random Forest']['eval']['accuracy']
svm_acc = results['SVM']['eval']['accuracy']

print(f"""
════════════════════════════════════════════════════════════════════════════════
                        CONCLUSIONS ET INTERPRÉTATIONS
════════════════════════════════════════════════════════════════════════════════

📊 RÉSULTATS OBTENUS:
   ┌────────────────────────┬────────────┬────────────┐
   │ Modèle                 │ Accuracy   │ F1 Weighted│
   ├────────────────────────┼────────────┼────────────┤
   │ Random Forest          │ {rf_acc:.1%}     │ {results['Random Forest']['eval']['f1_weighted']:.4f}     │
   │ SVM (RBF)              │ {svm_acc:.1%}     │ {results['SVM']['eval']['f1_weighted']:.4f}     │
   │ Logistic Regression    │ {lr_acc:.1%}     │ {results['Logistic Regression']['eval']['f1_weighted']:.4f}     │
   └────────────────────────┴────────────┴────────────┘

════════════════════════════════════════════════════════════════════════════════
🔍 INTERPRÉTATIONS:
════════════════════════════════════════════════════════════════════════════════

1️⃣  PERFORMANCE GLOBALE:
    • Random Forest domine avec {rf_acc:.1%} d'accuracy
    • C'est {rf_acc/0.25:.1f}x mieux que le hasard (25% pour 4 classes)
    → Les features extraites contiennent de l'information discriminante

2️⃣  LOGISTIC REGRESSION SOUS-PERFORME (~50%):
    • Performance proche du hasard pour un problème à 4 classes
    → Les classes ne sont PAS linéairement séparables dans l'espace des 5 features
    → Confirme la nécessité de modèles non-linéaires (RF, SVM RBF)

3️⃣  ÉCART CV vs TEST FAIBLE (~0.5%):
    • Pas d'overfitting majeur détecté
    → Les modèles généralisent correctement sur des données non vues

4️⃣  F1 > ACCURACY POUR LOGISTIC REGRESSION:
    • Le F1 weighted (0.51) > Accuracy (0.49)
    → Le class_weight='balanced' aide à mieux gérer les classes minoritaires

════════════════════════════════════════════════════════════════════════════════
⚠️  LIMITES DE L'APPROCHE:
════════════════════════════════════════════════════════════════════════════════

   • {rf_acc:.0%} reste INSUFFISANT pour un diagnostic médical fiable
   
   Les 5 features statistiques globales (mean, std, entropy...) ne capturent pas:
   ❌ La LOCALISATION des opacités pulmonaires
   ❌ Les PATTERNS SPATIAUX caractéristiques du COVID (ground-glass opacity)
   ❌ Les STRUCTURES ANATOMIQUES (poumons, médiastin, côtes)

════════════════════════════════════════════════════════════════════════════════
📋 RECOMMANDATIONS POUR AMÉLIORER:
════════════════════════════════════════════════════════════════════════════════

   1. FEATURES SPATIALES:
      • HOG (Histogram of Oriented Gradients) - capture les contours locaux
      • LBP (Local Binary Patterns) - capture les textures locales
   
   2. DEEP LEARNING:
      • CNN pré-entraîné (ResNet, VGG, EfficientNet)
      • Transfer learning depuis ImageNet
   
   3. SEGMENTATION:
      • Utiliser les masques pulmonaires disponibles dans data/*/masks/
      • Focaliser l'analyse sur les régions pulmonaires uniquement

════════════════════════════════════════════════════════════════════════════════
✅ Rapports sauvegardés dans: {REPORTS_DIR}
✅ Modèles sauvegardés dans: {MODELS_DIR}
════════════════════════════════════════════════════════════════════════════════
""")